In [5]:
from torch.testing._internal.common_distributed import spawn_threads_and_init_comms

In [6]:
from torch.testing._internal.common_distributed import spawn_threads_and_init_comms

import os
import torch
import os
from torch import distributed as dist
import torch.nn as nn
from torch.distributed.device_mesh import init_device_mesh
# from torch.distributed.fsdp import FullyShardedDataParallel as FSDP, ShardingStrategy
from torch.distributed._composable.fsdp import fully_shard
from torch.distributed.tensor.parallel import parallelize_module, ColwiseParallel, RowwiseParallel, SequenceParallel, PrepareModuleInput
from torch.distributed._tensor import Shard, Replicate, distribute_tensor, DTensor

# from llama2_model import Transformer, ModelArgs

import torch.distributed as dist
import torch
from torch.distributed._tensor import DeviceMesh, Shard, Replicate, distribute_tensor


from torch.distributed._composable.fsdp import fully_shard

In [7]:
import torch.distributed as dist
import torch
from torch.distributed._tensor import DeviceMesh, Shard, Replicate, distribute_tensor


In [8]:
from torch.distributed._composable.fsdp import fully_shard

In [24]:
@spawn_threads_and_init_comms
def shard_big_tensor(world_size):
    
    device= 'cpu'
    simple_llama2_config = ModelArgs(dim=4, n_layers=1, n_heads=4, vocab_size=8)

    model = Transformer.from_model_args(simple_llama2_config).to(device)
    model.tok_embeddings.weight = nn.Parameter(torch.arange(1.,33.).reshape(8,4))


    mesh = init_device_mesh('cpu', (2,1, 2), mesh_dim_names=["DDP","FSDP", "TP"])
#     print(mesh)

#     print(f' RANK {dist.get_rank()} {mesh['DDP', 'FSDP']} ')
#     print(f' RANK {dist.get_rank()} {mesh['FSDP']} ')
    
    
    layer_tp_plan = {
        # Now the input and output of SequenceParallel has Shard(1) layouts,
        # to represent the input/output tensors sharded on the sequence dimension
        "attention_norm": SequenceParallel(),
        # "attention": PrepareModuleInput(
        #     input_layouts=(Shard(1), Replicate()),
        #     desired_input_layouts=(Replicate(), Replicate()),
        # ),
        "attention.wq": ColwiseParallel(use_local_output=False),
        "attention.wk": ColwiseParallel(use_local_output=False),
        "attention.wv": ColwiseParallel(use_local_output=False),
        "attention.wo": RowwiseParallel(output_layouts=Shard(1)),
        "ffn_norm": SequenceParallel(),
        "feed_forward": PrepareModuleInput(
            input_layouts=(Shard(1),),
            desired_input_layouts=(Replicate(),),
        ),
        "feed_forward.w1": ColwiseParallel(),
        "feed_forward.w2": RowwiseParallel(output_layouts=Shard(1)),
        "feed_forward.w3": ColwiseParallel(),
    }
    # Apply TP
    for layer_id, transformer_block in enumerate(model.layers):
        # layer_tp_plan = {...}  # i.e. the plan we just generated

        parallelize_module(
            module=transformer_block,
            device_mesh=mesh['TP'],
            parallelize_plan=layer_tp_plan,
        )

    model = parallelize_module(
        model,
        mesh['TP'],
        {
            "tok_embeddings": RowwiseParallel(
                input_layouts=Replicate(),
                output_layouts=Shard(1),
            ),
            "norm": SequenceParallel(),
            "output": ColwiseParallel(
                input_layouts=Shard(1),
                output_layouts=Replicate()
            ),
        }
    )
    
#     print(f'rank {dist.get_rank()} mesh {mesh['TP']} DDP mesh {mesh['DDP','FSDP']}')

    for layer in model.layers:
        fully_shard(layer, mesh=mesh['DDP','FSDP'])

    fully_shard(model, mesh=mesh['DDP','FSDP'])
    

    
#     print(f' RANK {dist.get_rank()} its mesh {mesh['DDP', 'FSDP'] } wq.weight {model.layers[0].attention.wq.weight.to_local().shape}')

#     if dist.get_rank() == 0:
    print(f' RANK {dist.get_rank()} its mesh {mesh['DDP','FSDP'] } wq.weight {model.layers[0].attention.wq.weight.to_local()}, ')
    full = model.layers[0].attention.wq.weight.full_tensor()
#     print('yolo', full.shape, full)

   

In [25]:
world_size = 4
shard_big_tensor(world_size)

 RANK 1 its mesh DeviceMesh('cpu', [[1], [3]], mesh_dim_names=('DDP', 'FSDP')) wq.weight tensor([[ 0.0227, -0.0034, -0.0062,  0.0211],
        [-0.0046, -0.0220, -0.0173,  0.0011]],
       grad_fn=<_ToTorchTensorBackward>), 
 RANK 2 its mesh DeviceMesh('cpu', [[0], [2]], mesh_dim_names=('DDP', 'FSDP')) wq.weight tensor([[ 0.0187,  0.0047, -0.0212,  0.0276],
        [ 0.0298, -0.0108,  0.0122, -0.0339]],
       grad_fn=<_ToTorchTensorBackward>), 
 RANK 0 its mesh DeviceMesh('cpu', [[0], [2]], mesh_dim_names=('DDP', 'FSDP')) wq.weight tensor([[ 0.0093, -0.0151,  0.0237, -0.0201],
        [ 0.0156,  0.0106,  0.0451,  0.0130]],
       grad_fn=<_ToTorchTensorBackward>), 
 RANK 3 its mesh DeviceMesh('cpu', [[1], [3]], mesh_dim_names=('DDP', 'FSDP')) wq.weight tensor([[-0.0069,  0.0010, -0.0055,  0.0024],
        [-0.0241, -0.0198, -0.0218,  0.0225]],
       grad_fn=<_ToTorchTensorBackward>), 


In [74]:
@spawn_threads_and_init_comms
def shard_big_tensor(world_size):
    
    device= 'cpu'


    simple_llama2_config = ModelArgs(dim=4, n_layers=1, n_heads=4, vocab_size=8)

    model = Transformer.from_model_args(simple_llama2_config).to(device)
    model.tok_embeddings.weight = nn.Parameter(torch.arange(1.,33.).reshape(8,4))
#     print(model)
    # model.init_weights()

    # """
    # mesh = init_device_mesh('cpu', (2, 4), mesh_dim_names=["FSDP", "TP"])

    # mesh = init_device_mesh('cpu', (2,), mesh_dim_names=["TP"])

    mesh = init_device_mesh('cpu', (2, 1, 2), mesh_dim_names=["DDP", "FSDP", "TP"])
    layer_tp_plan = {
        # Now the input and output of SequenceParallel has Shard(1) layouts,
        # to represent the input/output tensors sharded on the sequence dimension
        "attention_norm": SequenceParallel(),
        "attention": PrepareModuleInput(
            input_layouts=(Shard(1), Replicate()),
            desired_input_layouts=(Replicate(), Replicate()),
        ),
        "attention.wq": ColwiseParallel(use_local_output=False),
        "attention.wk": ColwiseParallel(use_local_output=False),
        "attention.wv": ColwiseParallel(use_local_output=False),
        "attention.wo": RowwiseParallel(output_layouts=Shard(1)),
        "ffn_norm": SequenceParallel(),
        "feed_forward": PrepareModuleInput(
            input_layouts=(Shard(1),),
            desired_input_layouts=(Replicate(),),
        ),
        "feed_forward.w1": ColwiseParallel(),
        "feed_forward.w2": RowwiseParallel(output_layouts=Shard(1)),
        "feed_forward.w3": ColwiseParallel(),
    }
    # Apply TP
    for layer_id, transformer_block in enumerate(model.layers):
        # layer_tp_plan = {...}  # i.e. the plan we just generated

        parallelize_module(
            module=transformer_block,
            device_mesh=mesh['TP'],
            parallelize_plan=layer_tp_plan,
        )

    model = parallelize_module(
        model,
        mesh['TP'],
        {
            "tok_embeddings": RowwiseParallel(
                input_layouts=Replicate(),
                # output_layouts=Shard(1),
            ),
            "norm": SequenceParallel(),
            "output": ColwiseParallel(
                input_layouts=Shard(1),
                output_layouts=Replicate()
            ),
        }
    )

    
    for layer in model.layers:
        fully_shard(layer, mesh=mesh['DDP','FSDP'])

    fully_shard(model, mesh=mesh['DDP','FSDP'])
    
#     for layer in model.layers:
#         fully_shard(layer, mesh=mesh['FSDP'])

#     fully_shard(model, mesh=mesh['FSDP'])
    

    
    x = torch.arange(1,17).reshape(1,16)
    # out = model.layers[0].feed_forward.w1(x)
    # out = model.layers[0].attention.wq(x)

    # import time
    # time.sleep(5)
    print(f'Global rank: {dist.get_rank()}, tok_embeddings {model.tok_embeddings.weight} \n\n OUTPUT:\n ')
    # print(f'Global rank: {dist.get_rank()}, \n\n OUTPUT:\n {model.tok_embeddings(x).shape}')



In [75]:
world_size = 4
shard_big_tensor(world_size)

Global rank: 0, tok_embeddings DTensor(local_tensor=tensor([[ 1.,  2.,  3.,  4.],
        [ 5.,  6.,  7.,  8.],
        [ 9., 10., 11., 12.],
        [13., 14., 15., 16.]]), device_mesh=DeviceMesh('cpu', [[[0, 1]], [[2, 3]]], mesh_dim_names=('DDP', 'FSDP', 'TP')), placements=(Replicate(), _StridedShard(dim=0, sf=2), Shard(dim=0))) 

 OUTPUT:
 
Global rank: 3, tok_embeddings DTensor(local_tensor=tensor([[17., 18., 19., 20.],
        [21., 22., 23., 24.],
        [25., 26., 27., 28.],
        [29., 30., 31., 32.]]), device_mesh=DeviceMesh('cpu', [[[0, 1]], [[2, 3]]], mesh_dim_names=('DDP', 'FSDP', 'TP')), placements=(Replicate(), _StridedShard(dim=0, sf=2), Shard(dim=0))) 

 OUTPUT:
 
Global rank: 2, tok_embeddings DTensor(local_tensor=tensor([[ 1.,  2.,  3.,  4.],
        [ 5.,  6.,  7.,  8.],
        [ 9., 10., 11., 12.],
        [13., 14., 15., 16.]]), device_mesh=DeviceMesh('cpu', [[[0, 1]], [[2, 3]]], mesh_dim_names=('DDP', 'FSDP', 'TP')), placements=(Replicate(), _StridedShard(dim=

In [67]:
world_size = 4
shard_big_tensor(world_size)

Global rank: 3, tok_embeddings DTensor(local_tensor=tensor([[25., 26., 27., 28.],
        [29., 30., 31., 32.]]), device_mesh=DeviceMesh('cpu', [[0, 1], [2, 3]], mesh_dim_names=('FSDP', 'TP')), placements=(_StridedShard(dim=0, sf=2), Shard(dim=0))) 

 OUTPUT:
 
Global rank: 0, tok_embeddings DTensor(local_tensor=tensor([[1., 2., 3., 4.],
        [5., 6., 7., 8.]]), device_mesh=DeviceMesh('cpu', [[0, 1], [2, 3]], mesh_dim_names=('FSDP', 'TP')), placements=(_StridedShard(dim=0, sf=2), Shard(dim=0))) 

 OUTPUT:
 
Global rank: 1, tok_embeddings DTensor(local_tensor=tensor([[17., 18., 19., 20.],
        [21., 22., 23., 24.]]), device_mesh=DeviceMesh('cpu', [[0, 1], [2, 3]], mesh_dim_names=('FSDP', 'TP')), placements=(_StridedShard(dim=0, sf=2), Shard(dim=0))) 

 OUTPUT:
 
Global rank: 2, tok_embeddings DTensor(local_tensor=tensor([[ 9., 10., 11., 12.],
        [13., 14., 15., 16.]]), device_mesh=DeviceMesh('cpu', [[0, 1], [2, 3]], mesh_dim_names=('FSDP', 'TP')), placements=(_StridedShard(di

# Let's fucking goooo

### Data loader

 - load data from huggingface hub using hf datasets
 - make a dataloader (shuffle data, set batch sizes and so on)
 

In [32]:
from datasets import load_dataset
data = load_dataset('CohleM/rlpr_test_small', split='train')

INFO:datasets:PyTorch version 2.7.1 available.


In [33]:
type(data)

datasets.arrow_dataset.Dataset

In [34]:
from torch.utils.data import DataLoader

In [35]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, path):
        self.data = load_dataset(path, split='train')
        
    def __len__(self):
        return len(self.data)
        
    def __getitem__(self, idx):
        question = self.data['prompt'][idx]
        answer = self.data['reward_model'][idx]
        return {'question' : question , 'answer' : answer }

In [36]:
cd = CustomDataset('CohleM/rlpr_test_small')

In [37]:
cd[:2]

{'question': [[{'content': 'A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think> <answer> answer here </answer>.',
    'role': 'system'},
   {'content': 'If the firm could reduce the average age of its inventory from 73 days, to 63 day, by how much would it reduce its dollar investment in working capital?',
    'role': 'user'}],
  [{'content': 'A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer. The reasoning process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., <think> re

### training algorithm



```python

mesh = init_device_mesh(3,2) # 3 DDP, 2 TP



train_dataloader = DataLoader(the_dataset, shuffle=True, batch_size=64)
for train_data in train_dataloader: # each train_data is of len batch_size=64
    # split the data into different 3 DDP groups

    complete_rollout = []
    step = 0
    
    
    for each_data in train_data:
        each_data = each_data.reapeat(rollout.n)
        
        # generate responses.
        responses = llm.generate(each_data)
        advantages = calculate_advantages(each_data, responses)
        complete_rollout.append(each_data, responses)
    
    old_llm = make_copy_of_current_llm()    
    for update in range(no_of_updates_per_ppo_step):
        minibatches = DataLoader(complete_rollout, batch_size=ppo_mini_batch_size) # no shuffle here
        for mini_batch in minibatches:
            # generate the logprobs 
            old_logprobs = old_llm(mini_batchl)
            logprobs = llm(mini_batch)
            ppo_loss = calculate_ppo_loss(old_logprobs, logprobs, advantages) # i.e ratio * adv, where ratio = logprobs/ old_logprobs
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
         
            
        
        
    # we're doing GRPO so for each question generate,  
    
    
```

### training algorithm



```python

mesh = init_device_mesh(3,2) # 3 DDP, 2 TP



train_dataloader = DataLoader(the_dataset, shuffle=True, batch_size=64)
for train_data in train_dataloader: # each train_data is of len batch_size=64
    # split the data into different 3 DDP groups

    
    step = 0
    
    
    # repeat the train_data X rollout.n = new_train_data
    # split the new_train_data into different 3 DDP groups processes, each will get (len(new_train_data) / DDP_size) data
    new_train_data = train_data.repeat(rollout.n)
        
    #split here, i.e new_train_data into 3 ddp groups,
    # we also need an estimate of how many data each group can process at a time, lets say it's vllm_mini_batch
    new_train_data = DataLoader(new_train_data, vllm_mini_batch)
    complete_rollout = []
    for vllm_batch in new_train_data:
        # generate responses.
        responses = llm.generate(vllm_batch)
        complete_rollout.append(vllm_batch,responses)
    
     
    
    # all_gather data into one process then caculate the return/advantages,
    # after gathering complete_rollout should be full.
    
    advantages = calculate_advantages(each_data, responses)
    complete_rollout.append(each_data, responses)
    
    
    old_llm = make_copy_of_current_llm()  # we only need model parameters for old_llm, and will have the same sharding strategy as the current llm i.e llm.
    
    for update in range(no_of_updates_per_ppo_step):
        minibatches = DataLoader(complete_rollout, batch_size=ppo_mini_batch_size) # no shuffle here, or maybe split the mini_batches 
        for mini_batch in minibatches:
            # generate the logprobs 
            old_logprobs = old_llm(mini_batch)
            logprobs = llm(mini_batch)
            ppo_loss = calculate_ppo_loss(old_logprobs, logprobs, advantages) # i.e ratio * adv, where ratio = logprobs/ old_logprobs
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
         
            
     # update the vllm model with our llm's weight.
    
        
    # we're doing GRPO so for each question generate,  
    
    
```

# PPO trainer

dummy

In [38]:
from dataclasses import dataclass

@dataclass
class Config:
    train_batch_size: int = 64
    rollout_n: int = 8
    mini_batch_size: int = 8
    model_name: str = 'Qwen/Qwen2.5-0.5B-Instruct'
    dp_size: int = 2
    tp_size: int = 2
        
config = Config()

In [53]:
from dataclasses import dataclass

@dataclass
class Config:
    train_batch_size: int = 64
    rollout_n: int = 8
    mini_batch_size: int = 8
    model_name: str = 'Qwen/Qwen2.5-0.5B-Instruct'
    dp_size: int = 2
    tp_size: int = 2
        
config = Config()


# from transformers import AutoTokenizer, AutoModel
from torch.distributed.device_mesh import init_device_mesh
from torch.distributed._composable.fsdp import fully_shard
from torch.distributed.tensor.parallel import parallelize_module, ColwiseParallel, RowwiseParallel, SequenceParallel, PrepareModuleInput
from llama2_model import Transformer, ModelArgs

class Worker:
    """
    This is the policy that we will be updating with each gradient update, we rollout using this policy's
    parameters, and we use the logprobs from this policy, we will also copy it's weights to make it old policy
    """
    
    def __init__(self, config):
        
#         self.tokenizer = AutoTokenizer.from_pretrained(config.model_name)
        device= 'cpu'
        
        # define model from huggingface later on
#         simple_llama2_config = ModelArgs(dim=4, n_layers=1, n_heads=4, vocab_size=8)
#         self.model = Transformer.from_model_args(simple_llama2_config).to(device)

        # first make a device mesh
        self.mesh = init_device_mesh(device,(config.dp_size, config.tp_size), mesh_dim_names=["FSDP", "TP"])

#         # shard model's parameters on tp axis
#         self.prepare_tp_model()

#         # shard model's parameter on dp axis
#         prepare_dp_model()

    def prepare_optimizer(self):
        if config.tp_size > 1:
            self.model = prepare_tp_model(self.model, self.mesh)
            
#             if dist.get_rank() == 0:
                
#                 print(f' rank : {dist.get_rank()} attention wq {self.model.layers[0].attention.wq.weight}')
        
        self.model = prepare_dp_model(self.model, self.mesh)
#         if dist.get_rank() == 0:
#             print(f' after dp model rank: {dist.get_rank()} attention wq {self.model.layers[0].attention.wq.weight}')
        
        
def prepare_tp_model(model, mesh):

    layer_tp_plan = {
        # Now the input and output of SequenceParallel has Shard(1) layouts,
        # to represent the input/output tensors sharded on the sequence dimension
        "attention_norm": SequenceParallel(),
        "attention": PrepareModuleInput(
            input_layouts=(Shard(1), Replicate()),
            desired_input_layouts=(Replicate(), Replicate()),
        ),
        "attention.wq": ColwiseParallel(use_local_output=False),
        "attention.wk": ColwiseParallel(use_local_output=False),
        "attention.wv": ColwiseParallel(use_local_output=False),
        "attention.wo": RowwiseParallel(output_layouts=Shard(1)),
        "ffn_norm": SequenceParallel(),
        "feed_forward": PrepareModuleInput(
            input_layouts=(Shard(1),),
            desired_input_layouts=(Replicate(),),
        ),
        "feed_forward.w1": ColwiseParallel(),
        "feed_forward.w2": RowwiseParallel(output_layouts=Shard(1)),
        "feed_forward.w3": ColwiseParallel(),
    }


    # Apply TP
    for layer_id, transformer_block in enumerate(model.layers):

        parallelize_module(
            module=transformer_block,
            device_mesh=mesh['TP'],
            parallelize_plan=layer_tp_plan,
        )

    model = parallelize_module(
        model,
        mesh['TP'],
        {
            "tok_embeddings": RowwiseParallel(
                input_layouts=Replicate(),
                output_layouts=Shard(1),
            ),
            "norm": SequenceParallel(),
            "output": ColwiseParallel(
                input_layouts=Shard(1),
                output_layouts=Replicate()
            ),
        }
    )
    
    return model
    
def prepare_dp_model(model, mesh):
    for layer in model.layers:
        fully_shard(layer, mesh=mesh['FSDP'])

    sharded_model = fully_shard(model, mesh=mesh['FSDP'])
    return sharded_model


class Actor(Worker):
    def __init__(self, config):
        super().__init__(config)
        device= 'cpu'
        
#         define model from huggingface later on
        simple_llama2_config = ModelArgs(dim=4, n_layers=1, n_heads=4, vocab_size=8)
        self.model = Transformer.from_model_args(simple_llama2_config).to(device)
        
        # actor will need optimizer
        self.prepare_optimizer()


class Rollout(Worker):
    def __init__(self, config):
        super().__init__(config)
            
        # init model using sgland ()
        self.prepare_env_var()

        if self.mesh['TP'].get_local_rank() == 0:
            # initialize the SGL engine
            # change this env var to 0, otherwise, it will block other gpu !=0 i.e in 2x2 mesh it blocks local_rank = 2
            os.environ["SGLANG_BLOCK_NONZERO_RANK_CHILDREN"] = "0"
            self.llm = Engine(
                model_path=config.model_name,
                dtype="bfloat16",
                tp_size=self.mesh["tp"].size(),
                mem_fraction_static=config.gpu_memory_utilization,
                enable_memory_saver=True,
                port=30000 + dist.get_rank()
            )
                
                
    def prepare_env_var(self):
        if "TORCHELASTIC_USE_AGENT_STORE" in os.environ.keys(): # remove the use of common store for communication
            del os.environ["TORCHELASTIC_USE_AGENT_STORE"]
        monkey_patch_torch_reductions()
        
        
        # THE reason for doing this is because, we'll store rollout worker's (sglang) weight in these TP group
        # otherwise, SGL will use all the available cuda devices.
        cuda_visible_devices = [None] * self.config.tp_size
        dist.all_gather_object(cuda_visible_devices, os.environ['LOCAL_RANK'], self.mesh['TP'].get_group())
        os.environ['CUDA_VISIBLE_DEVICES'] = ",".join(cuda_visible_devices)
        
  
class Trainer:
    def __init__(self, config):
        self.config = config
        self.actor = Actor(config)
#         self.rollout = Rollout(config)
        
    def train():
        pass
        
        
import os
from torch import optim
from torch.distributed.checkpoint.state_dict import (
    StateDictOptions, get_model_state_dict, get_state_dict
)

@spawn_threads_and_init_comms
def shard_big_tensor(world_size):
    
    # can't use os.environ['LOCAL_RANK'] in spawn_threads_and_init_comms so using dist.get_rank() which gives the
    # local rank, but instead we must to os.environ['LOCAL_RANK'] otherwise.
    
    local_rank = dist.get_rank()
#     local_rank = int(os.environ["LOCAL_RANK"])
    
    # if gpu set this
    # torch.cuda.set_device(local_rank)
    
    ppo_trainer = Trainer(config)
    
    dummy_var = [None] * world_size
    dist.all_gather_object(
        dummy_var,
        local_rank,
        ppo_trainer.actor.mesh['TP'].get_group()
    )
    
    mesh = ppo_trainer.actor.mesh
    model = ppo_trainer.actor.model
    optimizer = optim.AdamW(model.parameters(), lr=1e-6)
    
    options = StateDictOptions(full_state_dict=False, cpu_offload=True)
    state_dict = get_model_state_dict(
        model, options=options
    )

    for idx, (name, tensor) in enumerate(state_dict.items()):

#         print(name, tensor)
        if name == 'layers.0.attention.wq.weight':
            
            serialized_tensor = tensor.full_tensor() if isinstance(tensor, DTensor) else tensor
            serialized_tensors = [None] * mesh['TP'].size() if mesh['TP'].get_local_rank() == 0 else None
            
            dist.gather_object(serialized_tensor, serialized_tensors, group_dst=0, group=mesh['TP'].get_group())

        
            local_tensor = tensor._local_tensor if isinstance(tensor, DTensor) else tensor
            print(f"rank {dist.get_rank()} local wq shape: {local_tensor.shape}")
#             print(f' rank {dist.get_rank()}', name, serialized_tensor.shape)
        
    
#         assert model._is_root, "Only support root model offloading to CPU"
#         for handle in model._all_handles:
#             if handle._offload_params:
#                 continue
#             flat_param = handle.flat_param
#             print(flat_param , 'shape', flat_param.shape)
            
#     print( isinstance(ppo_trainer.actor.model, FSDPModule))
#     print('dummy var')
#     actor = Actor(config)
#     print(f'Global RANK {dist.get_rank()} wq shape: {ppo_trainer.actor.model.layers[0].attention.wq.weight.to_local().shape} good')
#     print(f'Global RANK {dist.get_rank()}  dummy_var : {state_dict} ')



In [54]:
# from transformers import AutoTokenizer, AutoModel
from torch.distributed.device_mesh import init_device_mesh
from torch.distributed._composable.fsdp import fully_shard
from torch.distributed.tensor.parallel import parallelize_module, ColwiseParallel, RowwiseParallel, SequenceParallel, PrepareModuleInput
from llama2_model import Transformer, ModelArgs

class Worker:
    """
    This is the policy that we will be updating with each gradient update, we rollout using this policy's
    parameters, and we use the logprobs from this policy, we will also copy it's weights to make it old policy
    """
    
    def __init__(self, config):
        
#         self.tokenizer = AutoTokenizer.from_pretrained(config.model_name)
        device= 'cpu'
        
        # define model from huggingface later on
#         simple_llama2_config = ModelArgs(dim=4, n_layers=1, n_heads=4, vocab_size=8)
#         self.model = Transformer.from_model_args(simple_llama2_config).to(device)

        # first make a device mesh
        self.mesh = init_device_mesh(device,(config.dp_size, config.tp_size), mesh_dim_names=["FSDP", "TP"])

#         # shard model's parameters on tp axis
#         self.prepare_tp_model()

#         # shard model's parameter on dp axis
#         prepare_dp_model()

    def prepare_optimizer(self):
        if config.tp_size > 1:
            self.model = prepare_tp_model(self.model, self.mesh)
            
#             if dist.get_rank() == 0:
                
#                 print(f' rank : {dist.get_rank()} attention wq {self.model.layers[0].attention.wq.weight}')
        
        self.model = prepare_dp_model(self.model, self.mesh)
#         if dist.get_rank() == 0:
#             print(f' after dp model rank: {dist.get_rank()} attention wq {self.model.layers[0].attention.wq.weight}')
        
        
def prepare_tp_model(model, mesh):

    layer_tp_plan = {
        # Now the input and output of SequenceParallel has Shard(1) layouts,
        # to represent the input/output tensors sharded on the sequence dimension
        "attention_norm": SequenceParallel(),
        "attention": PrepareModuleInput(
            input_layouts=(Shard(1), Replicate()),
            desired_input_layouts=(Replicate(), Replicate()),
        ),
        "attention.wq": ColwiseParallel(use_local_output=False),
        "attention.wk": ColwiseParallel(use_local_output=False),
        "attention.wv": ColwiseParallel(use_local_output=False),
        "attention.wo": RowwiseParallel(output_layouts=Shard(1)),
        "ffn_norm": SequenceParallel(),
        "feed_forward": PrepareModuleInput(
            input_layouts=(Shard(1),),
            desired_input_layouts=(Replicate(),),
        ),
        "feed_forward.w1": ColwiseParallel(),
        "feed_forward.w2": RowwiseParallel(output_layouts=Shard(1)),
        "feed_forward.w3": ColwiseParallel(),
    }


    # Apply TP
    for layer_id, transformer_block in enumerate(model.layers):

        parallelize_module(
            module=transformer_block,
            device_mesh=mesh['TP'],
            parallelize_plan=layer_tp_plan,
        )

    model = parallelize_module(
        model,
        mesh['TP'],
        {
            "tok_embeddings": RowwiseParallel(
                input_layouts=Replicate(),
                output_layouts=Shard(1),
            ),
            "norm": SequenceParallel(),
            "output": ColwiseParallel(
                input_layouts=Shard(1),
                output_layouts=Replicate()
            ),
        }
    )
    
    return model
    
def prepare_dp_model(model, mesh):
    for layer in model.layers:
        fully_shard(layer, mesh=mesh['FSDP'])

    sharded_model = fully_shard(model, mesh=mesh['FSDP'])
    return sharded_model

In [55]:
class Actor(Worker):
    def __init__(self, config):
        super().__init__(config)
        device= 'cpu'
        
#         define model from huggingface later on
        simple_llama2_config = ModelArgs(dim=4, n_layers=1, n_heads=4, vocab_size=8)
        self.model = Transformer.from_model_args(simple_llama2_config).to(device)
        
        # actor will need optimizer
        self.prepare_optimizer()

In [56]:


class Rollout(Worker):
    def __init__(self, config):
        super().__init__(config)
            
        # init model using sgland ()
        self.prepare_env_var()

        if self.mesh['TP'].get_local_rank() == 0:
            # initialize the SGL engine
            # change this env var to 0, otherwise, it will block other gpu !=0 i.e in 2x2 mesh it blocks local_rank = 2
            os.environ["SGLANG_BLOCK_NONZERO_RANK_CHILDREN"] = "0"
            self.llm = Engine(
                model_path=config.model_name,
                dtype="bfloat16",
                tp_size=self.mesh["tp"].size(),
                mem_fraction_static=config.gpu_memory_utilization,
                enable_memory_saver=True,
                port=30000 + dist.get_rank()
            )
                
                
    def prepare_env_var(self):
        if "TORCHELASTIC_USE_AGENT_STORE" in os.environ.keys(): # remove the use of common store for communication
            del os.environ["TORCHELASTIC_USE_AGENT_STORE"]
        monkey_patch_torch_reductions()
        
        
        # THE reason for doing this is because, we'll store rollout worker's (sglang) weight in these TP group
        # otherwise, SGL will use all the available cuda devices.
        cuda_visible_devices = [None] * self.config.tp_size
        dist.all_gather_object(cuda_visible_devices, os.environ['LOCAL_RANK'], self.mesh['TP'].get_group())
        os.environ['CUDA_VISIBLE_DEVICES'] = ",".join(cuda_visible_devices)
        
  

In [43]:
class Trainer:
    def __init__(self, config):
        self.config = config
        self.actor = Actor(config)
#         self.rollout = Rollout(config)
        
    def train():
        pass
        
        


In [63]:
import os
from torch import optim
from torch.distributed.checkpoint.state_dict import (
    StateDictOptions, get_model_state_dict, get_state_dict
)

@spawn_threads_and_init_comms
def shard_big_tensor(world_size):
    
    # can't use os.environ['LOCAL_RANK'] in spawn_threads_and_init_comms so using dist.get_rank() which gives the
    # local rank, but instead we must to os.environ['LOCAL_RANK'] otherwise.
    
    local_rank = dist.get_rank()
#     local_rank = int(os.environ["LOCAL_RANK"])
    
    # if gpu set this
    # torch.cuda.set_device(local_rank)
    
    ppo_trainer = Trainer(config)
    
    dummy_var = [None] * world_size
    dist.all_gather_object(
        dummy_var,
        local_rank,
        ppo_trainer.actor.mesh['TP'].get_group()
    )
    
    mesh = ppo_trainer.actor.mesh
    model = ppo_trainer.actor.model
    optimizer = optim.AdamW(model.parameters(), lr=1e-6)
    
    options = StateDictOptions(full_state_dict=False, cpu_offload=True)
    state_dict = get_model_state_dict(
        model, options=options
    )

    for idx, (name, tensor) in enumerate(state_dict.items()):

#         print(name, tensor)
        if name == 'layers.0.attention.wq.weight':
            
            serialized_tensor = tensor.full_tensor() if isinstance(tensor, DTensor) else tensor
            serialized_tensors = [None] * mesh['TP'].size() if mesh['TP'].get_local_rank() == 0 else None
            
            dist.gather_object(serialized_tensor, serialized_tensors, group_dst=0, group=mesh['TP'].get_group())

        
#             local_tensor = tensor._local_tensor if isinstance(tensor, DTensor) else tensor
            print(f"rank {dist.get_rank()} local wq shape: {serialized_tensor.shape}, {tensor}")
        
    
#         assert model._is_root, "Only support root model offloading to CPU"
#         for handle in model._all_handles:
#             if handle._offload_params:
#                 continue
#             flat_param = handle.flat_param
#             print(flat_param , 'shape', flat_param.shape)
            
#     print( isinstance(ppo_trainer.actor.model, FSDPModule))
#     print('dummy var')
#     actor = Actor(config)
#     print(f'Global RANK {dist.get_rank()} wq shape: {ppo_trainer.actor.model.layers[0].attention.wq.weight.to_local().shape} good')
#     print(f'Global RANK {dist.get_rank()}  dummy_var : {state_dict} ')



In [64]:
from torch.distributed.fsdp import CPUOffloadPolicy, FSDPModule
from torch.distributed.fsdp._runtime_utils import _lazy_init

In [65]:
world_size = 4
shard_big_tensor(world_size)

rank 3 local wq shape: torch.Size([4, 4]), DTensor(local_tensor=tensor([[ 0.0078, -0.0135, -0.0184, -0.0121]]), device_mesh=DeviceMesh('cpu', [[0, 1], [2, 3]], mesh_dim_names=('FSDP', 'TP')), placements=(_StridedShard(dim=0, sf=2), Shard(dim=0)))rank 0 local wq shape: torch.Size([4, 4]), DTensor(local_tensor=tensor([[0.0231, 0.0122, 0.0189, 0.0002]]), device_mesh=DeviceMesh('cpu', [[0, 1], [2, 3]], mesh_dim_names=('FSDP', 'TP')), placements=(_StridedShard(dim=0, sf=2), Shard(dim=0)))

rank 1 local wq shape: torch.Size([4, 4]), DTensor(local_tensor=tensor([[ 0.0142,  0.0016,  0.0072, -0.0262]]), device_mesh=DeviceMesh('cpu', [[0, 1], [2, 3]], mesh_dim_names=('FSDP', 'TP')), placements=(_StridedShard(dim=0, sf=2), Shard(dim=0)))
rank 2 local wq shape: torch.Size([4, 4]), DTensor(local_tensor=tensor([[ 0.0020,  0.0222,  0.0368, -0.0408]]), device_mesh=DeviceMesh('cpu', [[0, 1], [2, 3]], mesh_dim_names=('FSDP', 'TP')), placements=(_StridedShard(dim=0, sf=2), Shard(dim=0)))


In [ ]:
import os

import torch
from torch import distributed as dist
from torch.distributed.device_mesh import init_device_mesh
# import sglang as sgl
from sglang.srt.entrypoints.engine import Engine
from sglang.srt.patch_torch import monkey_patch_torch_reductions

def setup():
        
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    backend = 'nccl' if device == 'cuda' else 'gloo'
    rank = int(os.environ["RANK"])
    world_size = int(os.environ["WORLD_SIZE"])
    local_rank = int(os.environ["LOCAL_RANK"])
    if device == 'cuda':
        torch.cuda.set_device(local_rank)
        
    # Initialize with explicit parameters
    dist.init_process_group(
        backend=backend, 
        world_size=world_size,
        rank=rank
    )
def prepare_environment_variables(mesh):

    if "TORCHELASTIC_USE_AGENT_STORE" in os.environ.keys():
        del os.environ["TORCHELASTIC_USE_AGENT_STORE"]
    monkey_patch_torch_reductions()
    cuda_visible_devices = mesh["TP"].size() * [None]
    dist.all_gather_object(
        cuda_visible_devices,
        os.environ["LOCAL_RANK"],
        mesh["TP"].get_group()
    )
    print(cuda_visible_devices)
    os.environ["CUDA_VISIBLE_DEVICES"] = ",".join(cuda_visible_devices)


def start():
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    local_rank = int(os.environ["LOCAL_RANK"])
    torch.cuda.set_device(local_rank)
    
    mesh = init_device_mesh(device, (2,2), mesh_dim_names = ['FSDP', 'TP'])
    print(f"Global rank {dist.get_rank()}, local rank {os.environ['LOCAL_RANK']} mesh {mesh['TP']} \n\n  ")
    

    engine = Engine(model_path='Qwen/Qwen2.5-0.5B-Instruct', mem_fraction_static=0.8, enable_memory_saver=True,port=30000 + dist.get_rank() )
print(torch.cuda.device_count())
    # if mesh['TP'].get_local_rank() == 0:
    #     prepare_environment_variables(mesh)
    #     os.environ["SGLANG_BLOCK_NONZERO_RANK_CHILDREN"] = "0"
    #     engine = Engine(model_path='Qwen/Qwen2.5-0.5B-Instruct', tp_size=2, mem_fraction_static=0.8, enable_memory_saver=True,port=30000 + dist.get_rank() )
    #     print('I believe done')
    # dist.barrier()

        

def main():
    setup()
    start()
    dist.destroy_process_group()
    

main()

In [ ]:
### TEST

In [ ]:
import os
import torch
from torch import distributed as dist
from torch.distributed.device_mesh import init_device_mesh
from sglang.srt.entrypoints.engine import Engine
import sglang as sgl
import nest_asyncio


from sglang.srt.patch_torch import monkey_patch_torch_reductions


def setup():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    backend = 'nccl' if device == 'cuda' else 'gloo'
    rank = int(os.environ["RANK"])
    world_size = int(os.environ["WORLD_SIZE"])
    local_rank = int(os.environ["LOCAL_RANK"])
    if device == 'cuda':
        torch.cuda.set_device(local_rank)
    os.environ["NCCL_CUMEM_ENABLE"] = "0"
    os.environ["NCCL_NVLS_ENABLE"] = "0"
    # Initialize with explicit parameters
    dist.init_process_group(
        backend=backend, 
        world_size=world_size,
        rank=rank
    )

def prepare_environment_variables(mesh):
    if "TORCHELASTIC_USE_AGENT_STORE" in os.environ.keys():
        del os.environ["TORCHELASTIC_USE_AGENT_STORE"]
    monkey_patch_torch_reductions()
    cuda_visible_devices = mesh["TP"].size() * [None]
    dist.all_gather_object(
        cuda_visible_devices,
        os.environ["LOCAL_RANK"],
        mesh["TP"].get_group()
    )
    # print(f' GLOBAL RNAK {dist.get_rank()} devices {cuda_visible_devices} ')
    os.environ["CUDA_VISIBLE_DEVICES"] = ",".join(cuda_visible_devices)

def start():
    # nest_asyncio.apply()

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    local_rank = int(os.environ["LOCAL_RANK"])
    torch.cuda.set_device(local_rank)
    torch.cuda.synchronize()
    mesh = init_device_mesh(device, (2,2), mesh_dim_names = ['FSDP', 'TP'])
    
    prepare_environment_variables(mesh)
    
    print(f"Global rank {dist.get_rank()}, local rank {os.environ['LOCAL_RANK']} mesh {mesh['TP']} visible devices {os.environ["CUDA_VISIBLE_DEVICES"]}\n\n  ")
    # print(torch.cuda.device_count())
    if mesh["TP"].get_local_rank() == 0:
        os.environ["SGLANG_BLOCK_NONZERO_RANK_CHILDREN"] = "0"
        llm = Engine(
                model_path="qwen/qwen2.5-0.5b-instruct",
                dtype="bfloat16",
                tp_size=mesh["TP"].size(),
                mem_fraction_static=0.5,
                enable_memory_saver=True,
                port=30000 + dist.get_rank()
            )
    dist.barrier()

    
    # Create engine on all ranks - SGLang will handle internal coordination
    # llm = Engine(
    #                 model_path="qwen/qwen2.5-0.5b-instruct",
    #                 dtype="bfloat16",
    #                 tp_size=4,
    #                 mem_fraction_static=0.5,
    #                 enable_memory_saver=True,
    #             )
    # llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct", tp_size=1,  dtype="bfloat16")
    # dist.barrier()
    print(f'Engine created on rank {dist.get_rank()}')

    return

        
def main():
    setup()
    start()
    dist.destroy_process_group()


if __name__ == '__main__':
    main()

In [ ]:
## actor model

In [ ]:

import os
import torch
from torch import distributed as dist
from torch.distributed.device_mesh import init_device_mesh
from sglang.srt.entrypoints.engine import Engine
import sglang as sgl
import nest_asyncio
from sglang.test.test_utils import DEFAULT_SMALL_MODEL_NAME_FOR_TEST
from transformers import AutoModel
from sglang.srt.patch_torch import monkey_patch_torch_reductions
from transformers import AutoModelForCausalLM

from torch.distributed._composable.fsdp import fully_shard
from torch.distributed._tensor import (DTensor, Replicate, Shard,
                                       distribute_tensor)
from torch.distributed.device_mesh import init_device_mesh
from torch.distributed.tensor.parallel import (ColwiseParallel,
                                               PrepareModuleInput,
                                               RowwiseParallel,
                                               SequenceParallel,
                                               parallelize_module)

def setup():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    backend = 'nccl' if device == 'cuda' else 'gloo'
    rank = int(os.environ["RANK"])
    world_size = int(os.environ["WORLD_SIZE"])
    local_rank = int(os.environ["LOCAL_RANK"])
    if device == 'cuda':
        torch.cuda.set_device(local_rank)
    os.environ["NCCL_CUMEM_ENABLE"] = "0"
    os.environ["NCCL_NVLS_ENABLE"] = "0"
    # Initialize with explicit parameters
    dist.init_process_group(
        backend=backend, 
        world_size=world_size,
        rank=rank
    )

def prepare_llama_tp_layer(layer, device_mesh):

    parallelize_plan = {
        "input_layernorm": SequenceParallel(),
        "self_attn.q_proj": ColwiseParallel(),
        "self_attn.k_proj": ColwiseParallel(),
        "self_attn.v_proj": ColwiseParallel(),
        "self_attn.o_proj": RowwiseParallel(
            output_layouts=Shard(1)
        ),
        "post_attention_layernorm": SequenceParallel(),
        "mlp.gate_proj": ColwiseParallel(),
        "mlp.up_proj": ColwiseParallel(),
        "mlp.down_proj": RowwiseParallel(
            output_layouts=Shard(1)
        )
    }
    parallelize_module(
        module=layer,
        device_mesh=device_mesh,
        parallelize_plan=parallelize_plan
    )

def prepare_environment_variables(mesh):
    if "TORCHELASTIC_USE_AGENT_STORE" in os.environ.keys():
        del os.environ["TORCHELASTIC_USE_AGENT_STORE"]
    monkey_patch_torch_reductions()
    cuda_visible_devices = mesh["TP"].size() * [None]
    dist.all_gather_object(
        cuda_visible_devices,
        os.environ["LOCAL_RANK"],
        mesh["TP"].get_group()
    )
    # print(f' GLOBAL RNAK {dist.get_rank()} devices {cuda_visible_devices} ')
    os.environ["CUDA_VISIBLE_DEVICES"] = ",".join(cuda_visible_devices)

def start():
    # nest_asyncio.apply()
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    local_rank = dist.get_rank()
    torch.cuda.set_device(local_rank)
    # torch.cuda.synchronize()
    mesh = init_device_mesh(device, (1,2,2), mesh_dim_names = ['DDP','FSDP', 'TP'])

    print(mesh)
    # -- rollout ---
    # prepare_environment_variables(mesh)
    # -- rollout ---
    
    # print(f"Global rank {dist.get_rank()}, local rank {os.environ['LOCAL_RANK']} mesh {mesh['TP']} visible devices {os.environ["CUDA_VISIBLE_DEVICES"]}\n\n  ")
    # print(torch.cuda.device_count())

    model_name = "ibm-granite/granite-3.3-2b-base"

    # ------- ROLLOUT ---------
    # if mesh["TP"].get_local_rank() == 0:
    #     os.environ["SGLANG_BLOCK_NONZERO_RANK_CHILDREN"] = "0"
    #     engine = Engine(
    #             model_path=model_name,
    #             dtype="bfloat16",
    #             tp_size=mesh["TP"].size(),
    #             mem_fraction_static=0.5,
    #             # enable_memory_saver=True,
    #             port=30000 + dist.get_rank()
    #         )

    #     # print(engine)
    #     param_name = 'model.layers.0.self_attn.q_proj.weight'
    #     # if dist.get_rank() == 0:
    #     #     breakpoint()

    #     reserved_gb = torch.cuda.memory_reserved() / (1024 ** 3)
    #     total_gb = torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)
    #     param = engine.get_weights_by_name(param_name)[0][:5]
    #     print(f"Global rank {dist.get_rank()}  param : {param} \n\n  ")
    # ------- ROLLOUT ---------


        
    # dist.barrier()

    

    # ----- rollout -----
    # Do the rollout
    # if mesh["TP"].get_local_rank() == 0:
    #     # release memory temporarily.
        # engine.release_memory_occupation()
    # ----- rollout -----


    # load the actor model
    model = AutoModelForCausalLM.from_pretrained(model_name)
    # print(model)
    for layer in model.model.layers:
        prepare_llama_tp_layer(layer, mesh['TP'])

    
# ----- outer block --------
    parallelize_plan = {
        "model.embed_tokens": ColwiseParallel(
            output_layouts=Shard(1)
        ),
        "model.norm": SequenceParallel(),
        "lm_head": ColwiseParallel()
    }
    parallelize_module(
        module=model,
        device_mesh=mesh['TP'],
        parallelize_plan=parallelize_plan
    )
# ----- outer block --------

# ----------FSDP -----------
    for layer in model.model.layers:
        fully_shard(layer, mesh=mesh['DDP', 'FSDP'])
    
    fully_shard(model, mesh=mesh['DDP', 'FSDP'])

    print(f' GLOBAL RANK {dist.get_rank()} {model.model.layers[0].self_attn.q_proj.weight.to_local().shape}')
    
    
    return

        
def main():
    setup()
    start()
    dist.destroy_process_group()


if __name__ == '__main__':
    main()

In [ ]:

import os
import torch
from torch import distributed as dist
from torch.distributed.device_mesh import init_device_mesh
from sglang.srt.entrypoints.engine import Engine
import sglang as sgl
import nest_asyncio
from sglang.test.test_utils import DEFAULT_SMALL_MODEL_NAME_FOR_TEST
from transformers import AutoModel
from sglang.srt.patch_torch import monkey_patch_torch_reductions
from transformers import AutoModelForCausalLM

from torch.distributed._composable.fsdp import fully_shard
from torch.distributed._tensor import (DTensor, Replicate, Shard,
                                       distribute_tensor)
from torch.distributed.device_mesh import init_device_mesh
from torch.distributed.tensor.parallel import (ColwiseParallel,
                                               PrepareModuleInput,
                                               RowwiseParallel,
                                               SequenceParallel,
                                               parallelize_module)


from torch.distributed.checkpoint.state_dict import (
    StateDictOptions, get_model_state_dict, get_state_dict
)


def setup():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    backend = 'nccl' if device == 'cuda' else 'gloo'
    rank = int(os.environ["RANK"])
    world_size = int(os.environ["WORLD_SIZE"])
    local_rank = int(os.environ["LOCAL_RANK"])
    if device == 'cuda':
        torch.cuda.set_device(local_rank)
    os.environ["NCCL_CUMEM_ENABLE"] = "0"
    os.environ["NCCL_NVLS_ENABLE"] = "0"
    # Initialize with explicit parameters
    dist.init_process_group(
        backend=backend, 
        world_size=world_size,
        rank=rank
    )

def prepare_llama_tp_layer(layer, device_mesh):

    parallelize_plan = {
        "input_layernorm": SequenceParallel(),
        "self_attn.q_proj": ColwiseParallel(),
        "self_attn.k_proj": ColwiseParallel(),
        "self_attn.v_proj": ColwiseParallel(),
        "self_attn.o_proj": RowwiseParallel(
            output_layouts=Shard(1)
        ),
        "post_attention_layernorm": SequenceParallel(),
        "mlp.gate_proj": ColwiseParallel(),
        "mlp.up_proj": ColwiseParallel(),
        "mlp.down_proj": RowwiseParallel(
            output_layouts=Shard(1)
        )
    }
    parallelize_module(
        module=layer,
        device_mesh=device_mesh,
        parallelize_plan=parallelize_plan
    )

def prepare_environment_variables(mesh):
    if "TORCHELASTIC_USE_AGENT_STORE" in os.environ.keys():
        del os.environ["TORCHELASTIC_USE_AGENT_STORE"]
    monkey_patch_torch_reductions()
    cuda_visible_devices = mesh["TP"].size() * [None]
    dist.all_gather_object(
        cuda_visible_devices,
        os.environ["LOCAL_RANK"],
        mesh["TP"].get_group()
    )
    # print(f' GLOBAL RNAK {dist.get_rank()} devices {cuda_visible_devices} ')
    os.environ["CUDA_VISIBLE_DEVICES"] = ",".join(cuda_visible_devices)

def start():
    # nest_asyncio.apply()
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    local_rank = dist.get_rank()
    torch.cuda.set_device(local_rank)
    # torch.cuda.synchronize()
    mesh = init_device_mesh(device, (1,2,2), mesh_dim_names = ['DDP','FSDP', 'TP'])

    print(mesh)
    # -- rollout ---
    # prepare_environment_variables(mesh)
    # -- rollout ---
    
    # print(f"Global rank {dist.get_rank()}, local rank {os.environ['LOCAL_RANK']} mesh {mesh['TP']} visible devices {os.environ["CUDA_VISIBLE_DEVICES"]}\n\n  ")
    # print(torch.cuda.device_count())

    model_name = "ibm-granite/granite-3.3-2b-base"

    # ------- ROLLOUT ---------
    # if mesh["TP"].get_local_rank() == 0:
    #     os.environ["SGLANG_BLOCK_NONZERO_RANK_CHILDREN"] = "0"
    #     engine = Engine(
    #             model_path=model_name,
    #             dtype="bfloat16",
    #             tp_size=mesh["TP"].size(),
    #             mem_fraction_static=0.5,
    #             # enable_memory_saver=True,
    #             port=30000 + dist.get_rank()
    #         )

    #     # print(engine)
    #     param_name = 'model.layers.0.self_attn.q_proj.weight'
    #     # if dist.get_rank() == 0:
    #     #     breakpoint()

    #     reserved_gb = torch.cuda.memory_reserved() / (1024 ** 3)
    #     total_gb = torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)
    #     param = engine.get_weights_by_name(param_name)[0][:5]
    #     print(f"Global rank {dist.get_rank()}  param : {param} \n\n  ")
    # ------- ROLLOUT ---------


        
    # dist.barrier()

    

    # ----- rollout -----
    # Do the rollout
    # if mesh["TP"].get_local_rank() == 0:
    #     # release memory temporarily.
        # engine.release_memory_occupation()
    # ----- rollout -----


    # load the actor model
    model = AutoModelForCausalLM.from_pretrained(model_name)
    # print(model)
    for layer in model.model.layers:
        prepare_llama_tp_layer(layer, mesh['TP'])

    
# ----- outer block --------
    parallelize_plan = {
        "model.embed_tokens": ColwiseParallel(
            output_layouts=Shard(1)
        ),
        "model.norm": SequenceParallel(),
        "lm_head": ColwiseParallel()
    }
    parallelize_module(
        module=model,
        device_mesh=mesh['TP'],
        parallelize_plan=parallelize_plan
    )
# ----- outer block --------

# ----------FSDP -----------
    for layer in model.model.layers:
        fully_shard(layer, mesh=mesh['DDP', 'FSDP'])
    
    fully_shard(model, mesh=mesh['DDP', 'FSDP'])

    # print(f' GLOBAL RANK {dist.get_rank()} {model.model.layers[0].self_attn.q_proj.weight.to_local().shape} memory allocated {torch.cuda.memory_allocated() / (1024 **3) }')

    # offload to cpu
    options = StateDictOptions(full_state_dict=False, cpu_offload=True)
    state_dict = get_model_state_dict(
        model, options=options
    )

    for idx, (name, tensor) in enumerate(state_dict.items()):
        # load to gpu again
        tensor = tensor.to(torch.cuda.current_device())
        # print(name)
        if name == 'model.layers.0.self_attn.q_proj.weight':
            
            serialized_tensor = tensor.full_tensor() if isinstance(tensor, DTensor) else tensor
            serialized_tensors = [None] * mesh['TP'].size() if mesh['TP'].get_local_rank() == 0 else None
            
            dist.gather_object(serialized_tensor, serialized_tensors, group_dst=0, group=mesh['TP'].get_group())

    
            print(f"rank {dist.get_rank()} seriliazed_tensor {serialized_tensor.shape} len_ST: {len(serialized_tensors) if isinstance(serialized_tensors,list) else serialized_tensors} ")
            
    
    
    return

        
def main():
    setup()
    start()
    dist.destroy_process_group()


if __name__ == '__main__':
    main()

Sglang weight update 

In [ ]:

import os
import torch
from torch import distributed as dist
from torch.distributed.device_mesh import init_device_mesh
from sglang.srt.entrypoints.engine import Engine
import sglang as sgl
import nest_asyncio
from sglang.test.test_utils import DEFAULT_SMALL_MODEL_NAME_FOR_TEST
from transformers import AutoModel
from sglang.srt.patch_torch import monkey_patch_torch_reductions
from transformers import AutoModelForCausalLM

from torch.distributed._composable.fsdp import fully_shard
from torch.distributed._tensor import (DTensor, Replicate, Shard,
                                       distribute_tensor)
from torch.distributed.device_mesh import init_device_mesh
from torch.distributed.tensor.parallel import (ColwiseParallel,
                                               PrepareModuleInput,
                                               RowwiseParallel,
                                               SequenceParallel,
                                               parallelize_module)


from torch.distributed.checkpoint.state_dict import (
    StateDictOptions, get_model_state_dict, get_state_dict
)


def setup():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    backend = 'nccl' if device == 'cuda' else 'gloo'
    rank = int(os.environ["RANK"])
    world_size = int(os.environ["WORLD_SIZE"])
    local_rank = int(os.environ["LOCAL_RANK"])
    if device == 'cuda':
        torch.cuda.set_device(local_rank)
    os.environ["NCCL_CUMEM_ENABLE"] = "0"
    os.environ["NCCL_NVLS_ENABLE"] = "0"
    # Initialize with explicit parameters
    dist.init_process_group(
        backend=backend, 
        world_size=world_size,
        rank=rank
    )

def prepare_llama_tp_layer(layer, device_mesh):

    parallelize_plan = {
        "input_layernorm": SequenceParallel(),
        "self_attn.q_proj": ColwiseParallel(),
        "self_attn.k_proj": ColwiseParallel(),
        "self_attn.v_proj": ColwiseParallel(),
        "self_attn.o_proj": RowwiseParallel(
            output_layouts=Shard(1)
        ),
        "post_attention_layernorm": SequenceParallel(),
        "mlp.gate_proj": ColwiseParallel(),
        "mlp.up_proj": ColwiseParallel(),
        "mlp.down_proj": RowwiseParallel(
            output_layouts=Shard(1)
        )
    }
    parallelize_module(
        module=layer,
        device_mesh=device_mesh,
        parallelize_plan=parallelize_plan
    )

def prepare_environment_variables(mesh):
    if "TORCHELASTIC_USE_AGENT_STORE" in os.environ.keys():
        del os.environ["TORCHELASTIC_USE_AGENT_STORE"]
    monkey_patch_torch_reductions()
    cuda_visible_devices = mesh["TP"].size() * [None]
    dist.all_gather_object(
        cuda_visible_devices,
        os.environ["LOCAL_RANK"],
        mesh["TP"].get_group()
    )
    # print(f' GLOBAL RNAK {dist.get_rank()} devices {cuda_visible_devices} ')
    os.environ["CUDA_VISIBLE_DEVICES"] = ",".join(cuda_visible_devices)

def start():
    # nest_asyncio.apply()
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    local_rank = dist.get_rank()
    torch.cuda.set_device(local_rank)
    # torch.cuda.synchronize()
    mesh = init_device_mesh(device, (1,2,2), mesh_dim_names = ['DDP','FSDP', 'TP'])

    # print(mesh)
    # -- rollout ---
    prepare_environment_variables(mesh)
    # -- rollout ---
    
    # print(f"Global rank {dist.get_rank()}, local rank {os.environ['LOCAL_RANK']} mesh {mesh['TP']} visible devices {os.environ["CUDA_VISIBLE_DEVICES"]}\n\n  ")
    # print(torch.cuda.device_count())

    model_name = "ibm-granite/granite-3.3-2b-base"

    # ------- ROLLOUT ---------
    if mesh["TP"].get_local_rank() == 0:
        os.environ["SGLANG_BLOCK_NONZERO_RANK_CHILDREN"] = "0"
        engine = Engine(
                model_path=model_name,
                dtype="bfloat16",
                tp_size=mesh["TP"].size(),
                mem_fraction_static=0.5,
                # enable_memory_saver=True,
                port=30000 + dist.get_rank()
            )

        # print(engine)
        param_name = 'model.layers.0.self_attn.q_proj.weight'
        # if dist.get_rank() == 0:
        #     breakpoint()

        # reserved_gb = torch.cuda.memory_reserved() / (1024 ** 3)
        # total_gb = torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)
        # param = engine.get_weights_by_name(param_name)[0][:5]
        # print(f"Global rank {dist.get_rank()}  param : {param} \n\n  ")
    # ------- ROLLOUT ---------

    print(f'gloabal rank {dist.get_rank()} memory allocated {torch.cuda.memory_allocated() }')
        
    # dist.barrier()

    

    # ----- rollout -----
    # Do the rollout
    if mesh["TP"].get_local_rank() == 0:
        # release memory temporarily.
        engine.release_memory_occupation()
        torch.cuda.empty_cache()
    # ----- rollout -----

    print(f'after release memory oocupation rank {dist.get_rank()} memory allocated {torch.cuda.memory_allocated() }')
    
    # load the actor model
    model = AutoModelForCausalLM.from_pretrained(model_name)
    # print(model)
    print(f'After automodel rank {dist.get_rank()} memory allocated {torch.cuda.memory_allocated() }')
    
    for layer in model.model.layers:
        prepare_llama_tp_layer(layer, mesh['TP'])

    
# ----- outer block --------
    parallelize_plan = {
        "model.embed_tokens": ColwiseParallel(
            output_layouts=Shard(1)
        ),
        "model.norm": SequenceParallel(),
        "lm_head": ColwiseParallel()
    }
    parallelize_module(
        module=model,
        device_mesh=mesh['TP'],
        parallelize_plan=parallelize_plan
    )
# ----- outer block --------

# ----------FSDP -----------
    for layer in model.model.layers:
        fully_shard(layer, mesh=mesh['DDP', 'FSDP'])
    
    fully_shard(model, mesh=mesh['DDP', 'FSDP'])

    # print(f' GLOBAL RANK {dist.get_rank()} {model.model.layers[0].self_attn.q_proj.weight.to_local().shape} memory allocated {torch.cuda.memory_allocated() / (1024 **3) }')

    # offload to cpu
    options = StateDictOptions(full_state_dict=False, cpu_offload=True)
    state_dict = get_model_state_dict(
        model, options=options
    )

    # -- rollout --
    # resume memory occupation
    torch.cuda.empty_cache()
    if mesh["TP"].get_local_rank() == 0:
        engine.resume_memory_occupation()


    # -- actor changes the model---
    # let's make some changes to the first and last row
    new_var = torch.tensor([1.5] * 5).to(torch.cuda.current_device())
    model.model.layers[0].self_attn.q_proj.weight[0][:5] = new_var
    model.model.layers[0].self_attn.q_proj.weight[-1][:5] = new_var
    
    print(model.model.layers[0].self_attn.q_proj.weight[0][:5])
        
    for idx, (name, tensor) in enumerate(state_dict.items()):
        # load to gpu again
        tensor = tensor.to(torch.cuda.current_device())
        # print(name)
        # if name == 'model.layers.0.self_attn.q_proj.weight':
            
        serialized_tensor = tensor.full_tensor() if isinstance(tensor, DTensor) else tensor
        serialized_tensors = [None] * mesh['TP'].size() if mesh['TP'].get_local_rank() == 0 else None
        
        dist.gather_object(serialized_tensor, serialized_tensors, group_dst=0, group=mesh['TP'].get_group())
        
        if mesh["TP"].get_local_rank() == 0:
            engine.update_weights_from_tensor(named_tensors=[(name, serialized_tensor)])

        
        print(f"rank {dist.get_rank()} seriliazed_tensor {serialized_tensor.shape} len_ST: {len(serialized_tensors) if isinstance(serialized_tensors,list) else serialized_tensors} ")

    dist.barrier()

    if mesh["TP"].get_local_rank() == 0:
        param_start = engine.get_weights_by_name(param_name)[0][:5]
        param_end = engine.get_weights_by_name(param_name)[-1][:5]
        print(f"Global rank {dist.get_rank()}  param after weight update : {param_start} and param end {param_end} \n\n  ")
            
            
    
    
    return

        
def main():
    setup()
    start()
    dist.destroy_process_group()


if __name__ == '__main__':
    main()

actor changes the model


In [ ]:
    # -- actor changes the model---
    # let's make some changes to the first and last row
    new_var = torch.tensor([1.5] * 5).to(torch.cuda.current_device())
    # print(model.model.layers[0].self_attn.q_proj.weight)
    # print(model.model.layers[0].self_attn.q_proj.weight[0][:5

    old_w = model.model.layers[0].self_attn.q_proj.weight
    local_weight = old_w.to_local().detach().clone()  
    local_weight[0][:5] = new_var

    new_mesh = old_w.device_mesh            # reuse original mesh
    placements = old_w.placements         # usually (Shard(0),)

    new_dt = DTensor.from_local(
        local_weight,
        device_mesh=new_mesh,
        placements=placements,
        run_check=False                    # safe because we kept the shape
    )

    model.model.layers[0].self_attn.q_proj.weight = torch.nn.Parameter(new_dt)

    print('after replacement', model.model.layers[0].self_attn.q_proj.weight)

# rollout and update works


In [ ]:

import os
import torch
from torch import distributed as dist
from torch.distributed.device_mesh import init_device_mesh
from sglang.srt.entrypoints.engine import Engine
import sglang as sgl
import nest_asyncio
from sglang.test.test_utils import DEFAULT_SMALL_MODEL_NAME_FOR_TEST
from transformers import AutoModel
from sglang.srt.patch_torch import monkey_patch_torch_reductions
from transformers import AutoModelForCausalLM

from torch.distributed._composable.fsdp import fully_shard
from torch.distributed._tensor import (DTensor, Replicate, Shard,
                                       distribute_tensor)
from torch.distributed.device_mesh import init_device_mesh
from torch.distributed.tensor.parallel import (ColwiseParallel,
                                               PrepareModuleInput,
                                               RowwiseParallel,
                                               SequenceParallel,
                                               parallelize_module)


from torch.distributed.checkpoint.state_dict import (
    StateDictOptions, get_model_state_dict, get_state_dict
)

from sglang.srt.utils import MultiprocessingSerializer
from sglang.srt.model_executor.model_runner import LocalSerializedTensor


def setup():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    backend = 'nccl' if device == 'cuda' else 'gloo'
    rank = int(os.environ["RANK"])
    world_size = int(os.environ["WORLD_SIZE"])
    local_rank = int(os.environ["LOCAL_RANK"])
    if device == 'cuda':
        torch.cuda.set_device(local_rank)
    os.environ["NCCL_CUMEM_ENABLE"] = "0"
    os.environ["NCCL_NVLS_ENABLE"] = "0"
    # Initialize with explicit parameters
    dist.init_process_group(
        backend=backend, 
        world_size=world_size,
        rank=rank
    )

def prepare_llama_tp_layer(layer, device_mesh):

    parallelize_plan = {
        "input_layernorm": SequenceParallel(),
        "self_attn.q_proj": ColwiseParallel(),
        "self_attn.k_proj": ColwiseParallel(),
        "self_attn.v_proj": ColwiseParallel(),
        "self_attn.o_proj": RowwiseParallel(
            output_layouts=Shard(1)
        ),
        "post_attention_layernorm": SequenceParallel(),
        "mlp.gate_proj": ColwiseParallel(),
        "mlp.up_proj": ColwiseParallel(),
        "mlp.down_proj": RowwiseParallel(
            output_layouts=Shard(1)
        )
    }
    parallelize_module(
        module=layer,
        device_mesh=device_mesh,
        parallelize_plan=parallelize_plan
    )

def prepare_environment_variables(mesh):
    if "TORCHELASTIC_USE_AGENT_STORE" in os.environ.keys():
        del os.environ["TORCHELASTIC_USE_AGENT_STORE"]
    monkey_patch_torch_reductions()
    cuda_visible_devices = mesh["TP"].size() * [None]
    dist.all_gather_object(
        cuda_visible_devices,
        os.environ["LOCAL_RANK"],
        mesh["TP"].get_group()
    )
    # print(f' GLOBAL RNAK {dist.get_rank()} devices {cuda_visible_devices} ')
    os.environ["CUDA_VISIBLE_DEVICES"] = ",".join(cuda_visible_devices)

def start():
    # nest_asyncio.apply()
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    local_rank = dist.get_rank()
    torch.cuda.set_device(local_rank)
    # torch.cuda.synchronize()
    mesh = init_device_mesh(device, (1,2,2), mesh_dim_names = ['DDP','FSDP', 'TP'])

    # print(mesh)
    # -- rollout ---
    prepare_environment_variables(mesh)
    # -- rollout ---
    
    # print(f"Global rank {dist.get_rank()}, local rank {os.environ['LOCAL_RANK']} mesh {mesh['TP']} visible devices {os.environ["CUDA_VISIBLE_DEVICES"]}\n\n  ")
    # print(torch.cuda.device_count())

    model_name = "ibm-granite/granite-3.3-2b-base"

    # ------- ROLLOUT ---------
    if mesh["TP"].get_local_rank() == 0:
        os.environ["SGLANG_BLOCK_NONZERO_RANK_CHILDREN"] = "0"
        engine = Engine(
                model_path=model_name,
                dtype="bfloat16",
                tp_size=mesh["TP"].size(),
                mem_fraction_static=0.5,
                # enable_memory_saver=True,
                port=30000 + dist.get_rank()
            )

        # print(engine)
        param_name = 'model.layers.0.self_attn.q_proj.weight'
        # if dist.get_rank() == 0:
        #     breakpoint()

        # reserved_gb = torch.cuda.memory_reserved() / (1024 ** 3)
        # total_gb = torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)
        # param = engine.get_weights_by_name(param_name)[0][:5]
        # print(f"Global rank {dist.get_rank()}  param : {param} \n\n  ")
    # ------- ROLLOUT ---------

    print(f'gloabal rank {dist.get_rank()} memory allocated {torch.cuda.memory_allocated() }')
        
    # dist.barrier()

    

    # ----- rollout -----
    # Do the rollout
    if mesh["TP"].get_local_rank() == 0:
        # release memory temporarily.
        engine.release_memory_occupation()
        torch.cuda.empty_cache()
    # ----- rollout -----

    print(f'after release memory oocupation rank {dist.get_rank()} memory allocated {torch.cuda.memory_allocated() }')
    
    # load the actor model
    model = AutoModelForCausalLM.from_pretrained(model_name)
    # print(model)
    print(f'After automodel rank {dist.get_rank()} memory allocated {torch.cuda.memory_allocated() }')
    
    for layer in model.model.layers:
        prepare_llama_tp_layer(layer, mesh['TP'])

    
# ----- outer block --------
    parallelize_plan = {
        "model.embed_tokens": ColwiseParallel(
            output_layouts=Shard(1)
        ),
        "model.norm": SequenceParallel(),
        "lm_head": ColwiseParallel()
    }
    parallelize_module(
        module=model,
        device_mesh=mesh['TP'],
        parallelize_plan=parallelize_plan
    )
# ----- outer block --------

# ----------FSDP -----------
    for layer in model.model.layers:
        fully_shard(layer, mesh=mesh['DDP', 'FSDP'])
    
    fully_shard(model, mesh=mesh['DDP', 'FSDP'])

    # print(f' GLOBAL RANK {dist.get_rank()} {model.model.layers[0].self_attn.q_proj.weight.to_local().shape} memory allocated {torch.cuda.memory_allocated() / (1024 **3) }')



    # -- rollout --
    # resume memory occupation
    torch.cuda.empty_cache()
    if mesh["TP"].get_local_rank() == 0:
        engine.resume_memory_occupation()


    # -- actor changes the model---
    # let's make some changes to the first and last row
    # -- actor changes the model---
    # let's make some changes to the first and last row
    new_var = torch.tensor([1.5] * 5).to(torch.cuda.current_device())
    # print(model.model.layers[0].self_attn.q_proj.weight)
    # print(model.model.layers[0].self_attn.q_proj.weight[0][:5

    old_w = model.model.layers[0].self_attn.q_proj.weight
    local_weight = old_w.to_local().detach().clone()  
    local_weight[0][:5] = new_var

    new_mesh = old_w.device_mesh            # reuse original mesh
    placements = old_w.placements         # usually (Shard(0),)

    new_dt = DTensor.from_local(
        local_weight,
        device_mesh=new_mesh,
        placements=placements,
        run_check=False                    # safe because we kept the shape
    )

    model.model.layers[0].self_attn.q_proj.weight = torch.nn.Parameter(new_dt)

    print('after replacement', model.model.layers[0].self_attn.q_proj.weight)


    # offload to cpu
    options = StateDictOptions(full_state_dict=False, cpu_offload=True)
    state_dict = get_model_state_dict(
        model, options=options
    )
    
    for idx, (name, tensor) in enumerate(state_dict.items()):
        # load to gpu again
        tensor = tensor.to(torch.cuda.current_device())
        # print(name)
        # if name == 'model.layers.0.self_attn.q_proj.weight':
            
        serialized_tensor = MultiprocessingSerializer.serialize(tensor.full_tensor() if isinstance(tensor, DTensor) else tensor)
        serialized_tensors = [None] * mesh['TP'].size() if mesh['TP'].get_local_rank() == 0 else None
        
        dist.gather_object(serialized_tensor, serialized_tensors, group_dst=0, group=mesh['TP'].get_group())
        
        if mesh["TP"].get_local_rank() == 0:
            # print(serialized_tensors)
            engine.update_weights_from_tensor(named_tensors=[(name, LocalSerializedTensor(values=serialized_tensors))])

        
        # print(f"rank {dist.get_rank()} seriliazed_tensor {serialized_tensor.shape} len_ST: {len(serialized_tensors) if isinstance(serialized_tensors,list) else serialized_tensors} ")

    dist.barrier()

    if mesh["TP"].get_local_rank() == 0:
        param_start = engine.get_weights_by_name(param_name)[0][:5]
        # param_end = engine.get_weights_by_name(param_name)[-1][:5]
        print(f"Global rank {dist.get_rank()}  param after weight update : {param_start} \n\n  ")
            
            

    return

        
def main():
    setup()
    start()
    dist.destroy_process_group()


if __name__ == '__main__':
    main()